In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'exploreReadWrite'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [ ]:
# read in data - csv
fil = "../data/students.csv"

schema = StructType([StructField('gender', StringType()),
                     StructField('race', StringType()),
                     StructField('parent_ed', StringType()),
                     StructField('lunch', StringType()),
                     StructField('test_prep', StringType()),
                     StructField('math_score', IntegerType()),
                     StructField('read_score', IntegerType()),
                     StructField('write_score', IntegerType())])

studentScores = spark.read.format('csv').options(header=True, inferschema=False).schema(schema).load(fil)#.repartition(8).cache()
print(studentScores.count())
studentScores.show(10)

In [ ]:
# read in data - parquet'
fil = '../data/users*.parquet' # the star tells spark to load them all simultaneously
users = spark.read.format('parquet').load(fil) # nothing about schema / headers needed
print(users.count())
users.show()

In [ ]:
# veiw some metadata
users.printSchema()
print(users.columns)

In [ ]:
# get datatype of a column
users.schema['salary'].dataType

In [ ]:
# summarize some columns
students.select([c for c in students.columns if c[-5:]=='score']).summary('min', 'mean', 'max').show()

In [ ]:
# mean by gender
students.select(['gender']+[c for c in students.columns if c[-5:]=='score']).groupBy('gender').\
    agg(mean('math_score').alias('Mean Math'), mean('read_score').alias('Mean Read'), mean('write_score').alias('Mean Write')).show()

In [ ]:
# write students data as parquet
fil = "../data/students"
students.write.format('parquet').mode('overwrite').save(fil)

In [ ]:
 # write students data as parquet - this time partitioned by gender
fil = "../data/studentsGend"
students.write.format('parquet').partitionBy('gender').mode('overwrite').save(fil)

In [ ]:
# finally, read in the gender-partitioned students parquet data
fil = "../data/studentsGend"
studentsP = spark.read.format('parquet').load(fil).repartition(2)
# verify the partitioning
print(studentsP.rdd.getNumPartitions())

In [ ]:
# counts by gender
studentsP.select('gender').groupBy('gender').agg(count('gender').alias('Gender Balance')).show()

In [6]:
''' create a dataframe '''
# get some data
n = 100
p = 5
randData = pd.DataFrame(np.random.rand(n, p+1))
randData[5] = randData[5] > 0.5
display(randData.head())
randData.values.tolist()

# define the schema
schem = StructType([StructField('col_%d'%i, FloatType(), False) for i in range(p)])
schem = schem.add(StructField('Flag', BooleanType(), False))

# create the dataframe
randData = spark.createDataFrame(randData, schema=schem).repartition('Flag')

# talk
randData.printSchema()
display(randData.limit(5).toPandas())

,0,1,2,3,4,5
0,0.107761,0.809760,0.102906,0.568573,0.618247,False
1,0.968840,0.690638,0.262231,0.765677,0.962609,True
2,0.880848,0.585071,0.342709,0.047035,0.038675,True
3,0.788106,0.572924,0.187714,0.436475,0.947135,True
4,0.166029,0.485552,0.675830,0.699974,0.843974,True


root
 |-- col_0: float (nullable = false)
 |-- col_1: float (nullable = false)
 |-- col_2: float (nullable = false)
 |-- col_3: float (nullable = false)
 |-- col_4: float (nullable = false)
 |-- Flag: boolean (nullable = false)



Py4JJavaError: An error occurred while calling o121.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 3) (192.168.150.128 executor driver): org.apache.spark.SparkException: 
Bad data in pyspark.daemon's standard output. Invalid port number:
  458961458 (0x1b5b3232)
Python command to execute the daemon was:
  ipython3 -m pyspark.daemon
Check that you don't have any unexpected modules or libraries in
your PYTHONPATH:
  /home/ahowe42/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/jars/spark-core_2.12-3.1.1.jar:/home/ahowe42/spark-3.1.1-bin-hadoop2.7/python:
Also, check if you have a sitecustomize.py module in your python path,
or in your python installation, that is printing to standard output
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemo

In [4]:
fil = "../data/randFlag"
randData.write.format('parquet').partitionBy('Flag').mode('overwrite').save(fil)

Py4JJavaError: An error occurred while calling o64.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.s

In [ ]:
sc.stop()